<a href="https://colab.research.google.com/github/AaruuC/SentimentAnalysis/blob/main/Copy_of_Project_Amazon_Fine_Food_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Some imports that you might need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import os
import tarfile

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


For the data, you can click download at
https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

You can choose to store the data in your google drive and mount your drive for accessing them from the following cell.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Or you can download it by the following cell:

In [ ]:
!pip install --upgrade --no-cache-dir gdown
if not os.path.exists("Reviews.csv"):
    !gdown 1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Downloading...
From (uriginal): https://drive.google.com/uc?id=1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk
From (redirected): https://drive.google.com/uc?id=1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk&confirm=t&uuid=8b49cd4d-e491-419d-b6e7-36a6e89bc953
To: /content/Reviews.csv
100% 301M/301M [00:07<00:00, 41.7MB/s]


In [ ]:
# Remeber to change the path here for the corresponding files you need
df = pd.read_csv('Reviews.csv')
print(df.shape)
df = df.head(1000)
print(df.shape)

(568454, 10)
(1000, 10)


In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
reviews = df[['Text', 'Score']]
reviews = reviews.drop_duplicates(subset='Text', keep='first')
reviews.dropna()

import re
import string

def process_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

reviews['Text'] = reviews['Text'].apply(process_text)


from nltk.tokenize import word_tokenize

reviews['Text'] = reviews['Text'].apply(word_tokenize)

from nltk.corpus import stopwords
def process_stopwords(text):
    return [word for word in text if not word in set(stopwords.words('english'))]

reviews['Text'] = reviews['Text'].apply(process_stopwords)


In [ ]:
X = reviews['Text']
y = reviews['Score']
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

X_train_flat = []
for sublist in X_train:
    sentence = ' '.join(sublist)
    X_train_flat.append(sentence)

X_test_flat = []
for sublist in X_test:
    sentence = ' '.join(sublist)
    X_test_flat.append(sentence)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 1500)
X_train = vectorizer.fit_transform(X_train_flat)
X_test = vectorizer.transform(X_test_flat)

In [ ]:
from sklearn import linear_model

classifier = linear_model.LogisticRegression()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
score1 = accuracy_score(y_test,y_pred)
score2 = f1_score(y_test,y_pred, average = None)
score3= recall_score(y_test,y_pred, average = None)
score4 = precision_score(y_test,y_pred, average=None)
score1_baseline = accuracy_score(y_test,[5 for _ in range(len(y_test))])
score2_baseline = f1_score(y_test,[5 for _ in range(len(y_test))], average = None)
score3_baseline= recall_score(y_test,[5 for _ in range(len(y_test))], average = None)
score4_baseline = precision_score(y_test,[5 for _ in range(len(y_test))], average=None)
print(score1, score2)
print(score1_baseline, score2_baseline)

Confusion Matrix:
 [[  5   2   3   3  14]
 [  4   0   0   3  14]
 [  0   0   3   1  19]
 [  2   0   2   5  26]
 [  3   1   1  21 168]]
0.6033333333333334 [0.24390244 0.         0.1875     0.14705882 0.77241379]
0.6466666666666666 [0.        0.        0.        0.        0.7854251]


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
dataset2 = pd.read_csv('./drive/MyDrive/womens_clothing.csv', header=0)
dataset2.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
reviews2 = dataset2[['Review Text', 'Rating']]
reviews2 = reviews2.drop_duplicates(subset='Review Text', keep='first')
reviews2.dropna()

import re
import string

def process_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text
reviews2['Review Text'] = reviews2['Review Text'].astype(str)
reviews2['Review Text'] = reviews2['Review Text'].apply(process_text)


from nltk.tokenize import word_tokenize

reviews2['Review Text'] = reviews2['Review Text'].apply(word_tokenize)

from nltk.corpus import stopwords
def process_stopwords(text):
    return [word for word in text if not word in set(stopwords.words('english'))]

reviews2['Review Text'] = reviews2['Review Text'].apply(process_stopwords)

In [ ]:
X_train = reviews['Text']
y_train = reviews['Score']
X_test = reviews2['Review Text']
y_test = reviews2['Rating']

X_train_flat = []
for sublist in X_train:
    sentence = ' '.join(sublist)
    X_train_flat.append(sentence)

X_test_flat = []
for sublist in X_test:
    sentence = ' '.join(sublist)
    X_test_flat.append(sentence)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 1500)
X_train = vectorizer.fit_transform(X_train_flat)
X_test = vectorizer.transform(X_test_flat)

In [ ]:
from sklearn import linear_model

classifier = linear_model.LogisticRegression()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
score1 = accuracy_score(y_test,y_pred)
score2 = f1_score(y_test,y_pred, average = None)
score3= recall_score(y_test,y_pred, average = None)
score4 = precision_score(y_test,y_pred, average=None)
score1_baseline = accuracy_score(y_test,[5 for _ in range(len(y_test))])
score2_baseline = f1_score(y_test,[5 for _ in range(len(y_test))], average = None)
score3_baseline= recall_score(y_test,[5 for _ in range(len(y_test))], average = None)
score4_baseline = precision_score(y_test,[5 for _ in range(len(y_test))], average=None)
print(score1, score2, score3, score4)
print(score1_baseline, score2_baseline, score3_baseline, score4_baseline)

Confusion Matrix:
 [[  111    31    86   111   482]
 [  146    59   161   287   895]
 [  177   102   325   542  1677]
 [  121   109   318   998  3362]
 [  135    98   320  1337 10645]]
0.5362491716368456 [0.14692257 0.06060606 0.16117034 0.24392032 0.71935397] [0.13520097 0.0381137  0.11512575 0.20334148 0.84922218] [0.16086957 0.14786967 0.26859504 0.30473282 0.62393764]
0.5537883808261542 [0.         0.         0.         0.         0.71282343] [0. 0. 0. 0. 1.] [0.         0.         0.         0.         0.55378838]


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install bert-tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
from bert import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
reviews_bert = df[['Text', 'Score']]
reviews_bert = reviews_bert.drop_duplicates(subset='Text', keep='first')
reviews_bert.dropna()
reviews_bert['Score'] = [1 if x >=3 else 0 for x in reviews_bert['Score']]
train_data, test_data = train_test_split(reviews_bert)
# train_data = np.asarray(train_data).astype(np.float32)

shifted_reviews_bert = dataset2[['Review Text', 'Rating']]
shifted_reviews_bert = shifted_reviews_bert.drop_duplicates(subset='Review Text', keep='first')
shifted_reviews_bert.dropna()
shifted_reviews_bert['Rating'] = [1 if x >=3 else 0 for x in shifted_reviews_bert['Rating']]


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

train_encodings = tokenizer(list(train_data['Text']), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data['Text']), truncation=True, padding=True)
test_shifted_encodings = tokenizer(list(shifted_reviews_bert['Review Text'].astype(str)), truncation=True, padding=True)

train_labels = torch.tensor(list(train_data['Score']))
test_labels = torch.tensor(list(test_data['Score']))
test_shifted_labels = torch.tensor(list(shifted_reviews_bert['Rating']))

train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), train_labels)
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']), test_labels)
test_shifted_dataset = torch.utils.data.TensorDataset(torch.tensor(test_shifted_encodings['input_ids']), torch.tensor(test_shifted_encodings['attention_mask']), test_shifted_labels)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    running_loss = 0.0
    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask=attention_mask)[0]
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    print('Epoch {}/{}: loss = {:.4f}'.format(epoch+1, 3, epoch_loss))

In [ ]:
import sklearn.metrics

model.eval()

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

total_loss = 0
num_batches = 0
total_preds = []
total_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        logits = model(input_ids, attention_mask=attention_mask)[0]
        loss = torch.nn.functional.cross_entropy(logits, labels)

        total_loss += loss.item()
        num_batches += 1

        preds = torch.argmax(logits, dim=1).tolist()
        total_preds.extend(preds)
        total_labels.extend(labels.tolist())

test_accuracy = sum([1 if total_preds[i] == total_labels[i] else 0 for i in range(len(total_preds))]) / len(total_preds)
test_f1 = sklearn.metrics.f1_score(total_labels, total_preds)
test_loss = total_loss / num_batches


print('Test Loss: {:.4f}'.format(test_loss))
print('Test Accuracy: {:.4f}'.format(test_accuracy))
print('Test F1 Score: {:.4f}'.format(test_f1))

In [ ]:
import sklearn.metrics

model.eval()

test_loader = torch.utils.data.DataLoader(test_shifted_dataset, batch_size=16, shuffle=False)

total_loss = 0
num_batches = 0
total_preds = []
total_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        logits = model(input_ids, attention_mask=attention_mask)[0]
        loss = torch.nn.functional.cross_entropy(logits, labels)

        total_loss += loss.item()
        num_batches += 1

        preds = torch.argmax(logits, dim=1).tolist()
        total_preds.extend(preds)
        total_labels.extend(labels.tolist())

test_accuracy = sum([1 if total_preds[i] == total_labels[i] else 0 for i in range(len(total_preds))]) / len(total_preds)
test_f1 = sklearn.metrics.f1_score(total_labels, total_preds)
test_loss = total_loss / num_batches


print('Test Loss: {:.4f}'.format(test_loss))
print('Test Accuracy: {:.4f}'.format(test_accuracy))
print('Test F1 Score: {:.4f}'.format(test_f1))

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, SimpleRNN
import keras.backend as K
from keras.callbacks import EarlyStopping

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
df = pd.read_csv('Reviews.csv')
df = df.head(1000)
reviews = df[['Text', 'Score']]
reviews['Score'] = reviews['Score'].map({1: -1, 2: -1, 3: 0, 4: 1, 5: 1})

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(reviews['Text'])
X = tokenizer.texts_to_sequences(reviews['Text'])
X = pad_sequences(X, maxlen=64, padding='post', truncating='post')

model = Sequential()
model.add(Embedding(5000, 64, input_length=64))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))

learning_rate = 0.01
batch_size = 16
epochs = 20

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_m])

early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X, reviews['Score'], batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[early_stop])

learning_rate = 0.001
epochs = 20
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_m])
history = model.fit(X, reviews['Score'], batch_size=batch_size, epochs=epochs, validation_split=0.2)

print(history.history)

In [ ]:
reviews2 = reviews2[['Review Text', 'Rating']]
reviews2['Rating'] = reviews2['Rating'].map({1: -1, 2: -1, 3: 0, 4: 1, 5: 1})

X2 = tokenizer.texts_to_sequences(reviews2['Review Text'])
X2 = pad_sequences(X2, maxlen=64, padding='post', truncating='post')

model.evaluate(X2, reviews2['Rating'])

In [ ]:
tf.random.set_seed(42)

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(reviews['Text'])
X = tokenizer.texts_to_sequences(reviews['Text'])
X = pad_sequences(X, maxlen=128, padding='post', truncating='post')

model = Sequential()
model.add(Embedding(10000, 128, input_length=128))
model.add(Bidirectional(LSTM(64, dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))


learning_rate = 0.005
batch_size = 16
epochs = 20


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_m])

early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X, reviews['Score'], batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[early_stop])


learning_rate = 0.001
epochs = 30
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_m])
history = model.fit(X, reviews['Score'], batch_size=batch_size, epochs=epochs, validation_split=0.2)


print(history.history)

In [ ]:
reviews2 = dataset2[['Review Text', 'Rating']]
reviews2 = reviews2.drop_duplicates(subset='Review Text', keep='first')
reviews2.dropna()
reviews2['Review Text'] = reviews2['Review Text'].astype(str)
reviews2['Review Text'] = reviews2['Review Text'].apply(process_text)


reviews2['Review Text'] = reviews2['Review Text'].apply(word_tokenize)

reviews2['Review Text'] = reviews2['Review Text'].apply(process_stopwords)
reviews2 = reviews2[['Review Text', 'Rating']]
reviews2['Rating'] = reviews2['Rating'].map({1: -1, 2: -1, 3: 0, 4: 1, 5: 1})
X2 = tokenizer.texts_to_sequences(reviews2['Review Text'])
X2 = pad_sequences(X2, maxlen=128, padding='post', truncating='post')

loss, f1 = model.evaluate(X2, reviews2['Rating'])